<a href="https://colab.research.google.com/github/paul3183/Pyspark_BigData_Udemy/blob/main/06_SECCION08_Spark_sql_avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SPARK SQL AVANZADO

In [1]:
# Instalar SDK Java 8

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark 3.4.3

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

# Descomprimir el archivo descargado de Spark

!tar xf spark-3.4.3-bin-hadoop3.tgz

# Establecer las variables de entorno

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

# Instalar la librería findspark

!pip install -q findspark

# Instalar pyspark

!pip install -q pyspark

# verificar la instalación ###

import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


##AGREGACIONES

### base de datos: Kaggle
### https://www.kaggle.com/datasets/usdot/flight-delays/data
#### contiene las cancelaciones y retrasos de vuelos nacionales en EEUU en el año 2015.

In [3]:
df = spark.read.parquet('/content/data/vuelos.parquet').limit(10)

In [ ]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [4]:
df.show(20, truncate=False)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

##FUNCIÓN COUNT, COUNTDISTINCT Y APROX_COUNT_DISTINCT

In [5]:
df = spark.read.parquet('/content/data02/dataframe/dataframe.parquet')

In [6]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- color: string (nullable = true)
 |-- cantidad: long (nullable = true)



In [7]:
df.show()

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  null| null|    1700|
|  null| rojo|    1300|
|  Juan| rojo|    1500|
+------+-----+--------+



In [8]:
#count:
#omite los valores nulos.
from pyspark.sql.functions import count

In [9]:
df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color')
).show()

+-------------+------------+
|conteo_nombre|conteo_color|
+-------------+------------+
|            2|           3|
+-------------+------------+



In [10]:
df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color'),
    count('*').alias('conteo_general')
).show()

+-------------+------------+--------------+
|conteo_nombre|conteo_color|conteo_general|
+-------------+------------+--------------+
|            2|           3|             4|
+-------------+------------+--------------+



In [11]:
#countDistinct:
#también omite los nulos y evita los repetidos:
from pyspark.sql.functions import countDistinct

df.select(
    countDistinct('color').alias('colores_diferentes')
).show()

+------------------+
|colores_diferentes|
+------------------+
|                 2|
+------------------+



In [12]:
#approx_count_distinct
from pyspark.sql.functions import approx_count_distinct

dataframe = spark.read.parquet('/content/data02/vuelos/vuelos.parquet')

In [15]:
dataframe.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [14]:
dataframe.select(
    countDistinct('AIRLINE').alias('valor_diferente_aerolinea'),
    approx_count_distinct('AIRLINE').alias('valor_diferente_aproximado')
).show()

+-------------------------+--------------------------+
|valor_diferente_aerolinea|valor_diferente_aproximado|
+-------------------------+--------------------------+
|                       14|                        13|
+-------------------------+--------------------------+



##FUNCIONES MIN Y MAX

In [16]:
vuelos = spark.read.parquet('/content/data02/vuelos/vuelos.parquet')

In [17]:
vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [18]:
from pyspark.sql.functions import min, max, col

In [19]:
vuelos.select(min('AIR_TIME').alias('menor_tiempo'), max('AIR_TIME').alias('mayor_tiempo')).show()

+------------+------------+
|menor_tiempo|mayor_tiempo|
+------------+------------+
|           7|         690|
+------------+------------+



In [20]:
vuelos.select(min('AIRLINE_DELAY').alias('minimo'), max('AIRLINE_DELAY').alias('maximo')).show()

+------+------+
|minimo|maximo|
+------+------+
|     0|  1971|
+------+------+



##FUNCIONES SUM, SUMDISTINCT, AVG

In [21]:
from pyspark.sql.functions import sum, sumDistinct, avg, count

In [22]:
#sum:
vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [24]:
vuelos.select(sum('DISTANCE').alias('sum_distance')).show()

+------------+
|sum_distance|
+------------+
|  4785357409|
+------------+



In [27]:
#sumDistinct:
vuelos.select(sumDistinct('DISTANCE').alias('suma_distancias_no_repetidas')).show()

+----------------------------+
|suma_distancias_no_repetidas|
+----------------------------+
|                     1442300|
+----------------------------+



In [28]:
#avg:
vuelos.select(avg('AIR_TIME').alias('tiempo_promedio_en_el_aire'),(sum('AIR_TIME') / count('AIR_TIME')).alias('promedio_manual')).show()


+--------------------------+------------------+
|tiempo_promedio_en_el_aire|   promedio_manual|
+--------------------------+------------------+
|        113.51162809012519|113.51162809012519|
+--------------------------+------------------+



##AGREGACIÓN CON AGRUPACIÓN

In [29]:
vuelos = spark.read.parquet('/content/data02/vuelos/vuelos.parquet')

In [30]:
vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [31]:
from pyspark.sql.functions import desc

In [32]:
vuelos.groupBy('ORIGIN_AIRPORT').count().orderBy(desc('count')).show()

+--------------+------+
|ORIGIN_AIRPORT| count|
+--------------+------+
|           ATL|346836|
|           ORD|285884|
|           DFW|239551|
|           DEN|196055|
|           LAX|194673|
|           SFO|148008|
|           PHX|146815|
|           IAH|146622|
|           LAS|133181|
|           MSP|112117|
|           MCO|110982|
|           SEA|110899|
|           DTW|108500|
|           BOS|107847|
|           EWR|101772|
|           CLT|100324|
|           LGA| 99605|
|           SLC| 97210|
|           JFK| 93811|
|           BWI| 86079|
+--------------+------+
only showing top 20 rows



In [33]:
vuelos.groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT').count().orderBy(desc('count')).show()

+--------------+-------------------+-----+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+-------------------+-----+
|           SFO|                LAX|13744|
|           LAX|                SFO|13457|
|           JFK|                LAX|12016|
|           LAX|                JFK|12015|
|           LAS|                LAX| 9715|
|           LGA|                ORD| 9639|
|           LAX|                LAS| 9594|
|           ORD|                LGA| 9575|
|           SFO|                JFK| 8440|
|           JFK|                SFO| 8437|
|           OGG|                HNL| 8313|
|           HNL|                OGG| 8282|
|           LAX|                ORD| 8256|
|           ATL|                LGA| 8234|
|           LGA|                ATL| 8215|
|           ATL|                MCO| 8202|
|           MCO|                ATL| 8202|
|           SFO|                LAS| 7995|
|           ORD|                LAX| 7941|
|           LAS|                SFO| 7870|
+----------

##Multiples agregaciones por grupo

In [34]:
vuelos = spark.read.parquet('/content/data02/vuelos/vuelos.parquet')

In [35]:
from pyspark.sql.functions import min, max, desc, avg

In [42]:
vuelos.groupBy('ORIGIN_AIRPORT').agg(count('AIR_TIME').alias('tiempo_en_el_aire'), min('AIR_TIME').alias('tiempo_minimo_en_el_aire'), max('AIR_TIME').alias('tiempo_maximo_en_el_aire')).orderBy(desc(col('tiempo_en_el_aire'))).show()

+--------------+-----------------+------------------------+------------------------+
|ORIGIN_AIRPORT|tiempo_en_el_aire|tiempo_minimo_en_el_aire|tiempo_maximo_en_el_aire|
+--------------+-----------------+------------------------+------------------------+
|           ATL|           343506|                      15|                     614|
|           ORD|           276554|                      13|                     571|
|           DFW|           232647|                      11|                     534|
|           DEN|           193402|                      12|                     493|
|           LAX|           192003|                      14|                     409|
|           PHX|           145552|                      19|                     444|
|           SFO|           145491|                       8|                     389|
|           IAH|           144019|                      15|                     524|
|           LAS|           131937|                      25|      

In [43]:
vuelos.groupBy('MONTH').agg(count('ARRIVAL_DELAY').alias('conteo_retrasos'), avg('DISTANCE').alias('distancia_promedio')).orderBy(desc('conteo_retrasos')).show()

+-----+---------------+------------------+
|MONTH|conteo_retrasos|distancia_promedio|
+-----+---------------+------------------+
|    7|         514384| 841.4772794487611|
|    8|         503956| 834.8244276603413|
|    6|         492847| 835.6302716626612|
|    3|         492138| 816.0553268611494|
|    5|         489641| 823.3230588760807|
|   10|         482878| 816.4436127652134|
|    4|         479251| 817.0060476016745|
|   12|         469717| 837.8018926194103|
|   11|         462367| 820.2482434846529|
|    9|         462153| 815.8487523282274|
|    1|         457013| 803.2612794913696|
|    2|         407663|  800.785449834689|
+-----+---------------+------------------+



##AGREGACIÓN CON PIVOTE

In [44]:
estudiantes = spark.read.parquet('/content/data03/estudiantes.parquet')

In [45]:
estudiantes.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- peso: long (nullable = true)
 |-- graduacion: long (nullable = true)



In [46]:
estudiantes.show()

+------+----+----+----------+
|nombre|sexo|peso|graduacion|
+------+----+----+----------+
|  Jose|   M|  80|      2000|
| Hilda|   F|  50|      2000|
|  Juan|   M|  75|      2000|
| Pedro|   M|  76|      2001|
|Katia+|   F|  65|      2001|
+------+----+----+----------+



In [47]:
from pyspark.sql.functions import min, max, avg, col

In [50]:
estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|graduacion|F_avg(peso)|F_min(peso)|F_max(peso)|M_avg(peso)|M_min(peso)|M_max(peso)|
+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|      2001|       65.0|         65|         65|       76.0|         76|         76|
|      2000|       50.0|         50|         50|       77.5|         75|         80|
+----------+-----------+-----------+-----------+-----------+-----------+-----------+



In [51]:
estudiantes.groupBy('graduacion').pivot('sexo', ['M']).agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+
|graduacion|M_avg(peso)|M_min(peso)|M_max(peso)|
+----------+-----------+-----------+-----------+
|      2001|       76.0|         76|         76|
|      2000|       77.5|         75|         80|
+----------+-----------+-----------+-----------+



In [52]:
estudiantes.groupBy('graduacion').pivot('sexo', ['F']).agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+
|graduacion|F_avg(peso)|F_min(peso)|F_max(peso)|
+----------+-----------+-----------+-----------+
|      2001|       65.0|         65|         65|
|      2000|       50.0|         50|         50|
+----------+-----------+-----------+-----------+



##JOINS

In [53]:
#Inner join:
empleados = spark.read.parquet('/content/data04/empleados/empleados.parquet')
departamentos = spark.read.parquet('/content/data04/departamentos/departamentos.parquet')

In [54]:
empleados.show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [55]:
departamentos.show()

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 31|     letras|
| 33|    derecho|
| 34| matemática|
| 35|informática|
+---+-----------+



In [56]:
from pyspark.sql.functions import col

In [57]:
join_df = empleados.join(departamentos, col('num_dpto') == col('id'))
join_df.show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [58]:
join_df = empleados.join(departamentos, col('num_dpto') == col('id'), 'inner')
join_df.show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [59]:
join_df = empleados.join(departamentos).where(col('num_dpto')==col('id'))
join_df.show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [60]:
#left outer join:
from pyspark.sql.functions import col

In [61]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'leftouter').show()

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Pedro|       0|null|       null|
| Laura|      34|  34| matemática|
|Sandro|      31|  31|     letras|
+------+--------+----+-----------+



In [62]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'left').show()

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Pedro|       0|null|       null|
| Laura|      34|  34| matemática|
|Sandro|      31|  31|     letras|
+------+--------+----+-----------+



In [63]:
#right outer join:
from pyspark.sql.functions import col

In [64]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'rightouter').show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|Sandro|      31| 31|     letras|
| Katia|      33| 33|    derecho|
|  Luis|      33| 33|    derecho|
| Laura|      34| 34| matemática|
|  Raul|      34| 34| matemática|
|  null|    null| 35|informática|
+------+--------+---+-----------+



In [65]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'right').show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|Sandro|      31| 31|     letras|
| Katia|      33| 33|    derecho|
|  Luis|      33| 33|    derecho|
| Laura|      34| 34| matemática|
|  Raul|      34| 34| matemática|
|  null|    null| 35|informática|
+------+--------+---+-----------+



In [66]:
#full outer join:
from pyspark.sql.functions import col

In [68]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'fullouter').show()

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
| Pedro|       0|null|       null|
|Sandro|      31|  31|     letras|
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Laura|      34|  34| matemática|
|  null|    null|  35|informática|
+------+--------+----+-----------+



In [69]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'outer').show()

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
| Pedro|       0|null|       null|
|Sandro|      31|  31|     letras|
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Laura|      34|  34| matemática|
|  null|    null|  35|informática|
+------+--------+----+-----------+



In [71]:
#Left anti join:
from pyspark.sql.functions import col

In [73]:
#registros de empleados que no tienen correspondencia en departamentos
empleados.join(departamentos, col('num_dpto') == col('id'), 'left_anti').show()

+------+--------+
|nombre|num_dpto|
+------+--------+
| Pedro|       0|
+------+--------+



In [74]:
departamentos.join(empleados, col('num_dpto') == col('id'), 'left_anti').show()

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 35|informática|
+---+-----------+



In [75]:
#Left semi join:
from pyspark.sql.functions import col

In [76]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'left_semi').show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [77]:
#Cross Join:
from pyspark.sql.functions import col

In [78]:
df = empleados.crossJoin(departamentos)

In [79]:
df.show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 31|     letras|
|  Luis|      33| 33|    derecho|
|  Luis|      33| 34| matemática|
|  Luis|      33| 35|informática|
| Katia|      33| 31|     letras|
| Katia|      33| 33|    derecho|
| Katia|      33| 34| matemática|
| Katia|      33| 35|informática|
|  Raul|      34| 31|     letras|
|  Raul|      34| 33|    derecho|
|  Raul|      34| 34| matemática|
|  Raul|      34| 35|informática|
| Pedro|       0| 31|     letras|
| Pedro|       0| 33|    derecho|
| Pedro|       0| 34| matemática|
| Pedro|       0| 35|informática|
| Laura|      34| 31|     letras|
| Laura|      34| 33|    derecho|
| Laura|      34| 34| matemática|
| Laura|      34| 35|informática|
+------+--------+---+-----------+
only showing top 20 rows



In [81]:
df.count()

24

##MANEJO DE NOMBRES DE COLUMNAS DUPLICADOS

In [82]:
from pyspark.sql.functions import col

In [83]:
depa = departamentos.withColumn('num_dpto', col('id'))

In [85]:
depa.printSchema()
empleados.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [87]:
empleados.join(depa, empleados['num_dpto'] == depa['num_dpto']).show()

+------+--------+---+-----------+--------+
|nombre|num_dpto| id|nombre_dpto|num_dpto|
+------+--------+---+-----------+--------+
|Sandro|      31| 31|     letras|      31|
| Katia|      33| 33|    derecho|      33|
|  Luis|      33| 33|    derecho|      33|
| Laura|      34| 34| matemática|      34|
|  Raul|      34| 34| matemática|      34|
+------+--------+---+-----------+--------+



In [88]:
df_con_duplicados = empleados.join(depa, empleados['num_dpto'] == depa['num_dpto'])

In [89]:
df_con_duplicados.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [91]:
df_con_duplicados.select(empleados['num_dpto']).show()

+--------+
|num_dpto|
+--------+
|      33|
|      33|
|      34|
|      34|
|      31|
+--------+



In [92]:
#usando columna de unión:
df2 = empleados.join(depa,'num_dpto')

In [93]:
df2.printSchema()

root
 |-- num_dpto: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



In [94]:
empleados.join(depa, ['num_dpto']).show()


+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+



In [95]:
empleados.join(depa, ['num_dpto']).printSchema()

root
 |-- num_dpto: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



#SHUFFLE HASH JOIN Y BROADCAST HASH JOIN

In [96]:
from pyspark.sql.functions import col, broadcast

In [97]:
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [98]:
#como sabemos si realmenete se hizo el broadcast?
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#4903L], [id#4906L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#4903L)
   :  +- FileScan parquet [nombre#4902,num_dpto#4903L] Batched: true, DataFilters: [isnotnull(num_dpto#4903L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/data04/empleados/empleados.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=3586]
      +- Filter isnotnull(id#4906L)
         +- FileScan parquet [id#4906L,nombre_dpto#4907] Batched: true, DataFilters: [isnotnull(id#4906L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/data04/departamentos/departamentos.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:string>


